In [ ]:
try:
    from google.colab import drive
    COLAB = True
    print("Note: using Google CoLab")
    %tensorflow_version 2.x
except:
    print("Note: not using Google CoLab")
    COLAB = False

Note: using Google CoLab
Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [ ]:
# HIDE OUTPUT
CMD = "wget https://raw.githubusercontent.com/Diyago/"\
  "GAN-for-tabular-data/master/requirements.txt"

##installing tabgan
##Tabular-DATA
!{CMD}
!pip install -r requirements.txt
!pip install tabgan

--2023-12-16 09:15:10--  https://raw.githubusercontent.com/Diyago/GAN-for-tabular-data/master/requirements.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 214 [text/plain]
Saving to: ‘requirements.txt.3’

requirements.txt.3  100%[===================>]     214  --.-KB/s    in 0s      

2023-12-16 09:15:10 (3.41 MB/s) - ‘requirements.txt.3’ saved [214/214]



In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# HIDE OUTPUT
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
import pandas as pd
import io
import os
import requests
import numpy as np
from sklearn import metrics

In [ ]:
df = pd.read_csv(
    "/content/drive/My Drive/Data.csv",
    na_values=['NA', '?'])

In [ ]:
df

,From Date,SL NO,PM2.5,PM10,NO,NO2,Nox,NH3,SO2,CO,...,Benzene,Eth-Benzene,MP-Xylene,WS,WD,SR,BP,AT,RF,Final
0,2019-02-18 00:00:00,1,58.04,81.06,1.81,6.400000,9.66,27.18,13.770000,0.49,...,0.92,NaN,0.33,1.86,229.44,0.220000,1002.190000,20.87,0.00,1
1,2019-02-18 01:00:00,2,60.00,82.39,1.91,6.080000,9.70,26.46,13.640000,0.44,...,0.84,NaN,0.29,1.54,217.20,1.460000,956.000000,18.92,0.24,1
2,2019-02-18 02:00:00,3,55.50,73.42,2.22,6.710000,10.86,27.06,13.640000,0.44,...,0.77,NaN,0.23,0.66,197.21,241.906008,1001.600000,18.33,0.00,1
3,2019-02-18 03:00:00,4,66.56,96.04,2.79,7.000000,12.10,26.34,13.720000,0.45,...,0.82,NaN,0.21,0.61,206.44,241.906008,1001.410000,17.63,0.00,1
4,2019-02-18 04:00:00,5,67.08,97.21,3.29,9.450000,15.53,25.16,13.800000,0.47,...,0.74,NaN,0.22,0.54,230.53,241.906008,980.670000,16.61,0.00,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33062,2022-12-30 20:00:00,33063,295.72,371.83,5.54,2.620000,5.41,16.13,13.910000,2.64,...,2.96,3.16,3.37,0.31,168.12,241.906008,1009.220000,15.96,0.00,1
33063,2022-12-30 21:00:00,33064,362.60,448.50,5.61,2.620000,5.46,17.45,13.880000,2.79,...,3.12,3.34,3.56,0.69,132.27,11.170000,967.180000,14.93,0.48,1
33064,2022-12-30 22:00:00,33065,277.86,353.78,5.52,2.460000,5.29,17.92,13.740000,3.25,...,3.64,3.89,4.15,0.36,173.64,241.906008,1009.110000,15.22,0.00,1
33065,2022-12-30 23:00:00,33066,380.00,450.00,5.46,2.470000,5.32,18.01,14.120000,3.28,...,3.68,3.94,4.21,0.82,163.19,9.570000,912.740000,13.45,1.09,1


In [ ]:
df = pd.read_csv(
    "/content/drive/My Drive/Data.csv",
    na_values=['NA', '?'])

#'SL NO' , 'PM2.5' , 'PM10' , 'NO' , 'NO2' , 'Nox' , 'NH3' , 'SO2' , 'CO' , 'Ozone' , 'Benzene' , 'Eth-Benzene' , 'MP-Xylene' , 'WS' , 'WD'	, 'SR'
COLS_USED = ['PM2.5', 'PM10', 'NO', 'NO2',
          'Nox', 'NH3', 'SO2','CO']
COLS_TRAIN = ['PM2.5', 'PM10', 'NO', 'NO2',
          'Nox', 'NH3', 'SO2','CO']

df = df[COLS_USED]

# Handle missing value
df['PM2.5'] = df['PM2.5'].fillna(df['PM2.5'].median())
df['PM10'] = df['PM10'].fillna(df['PM10'].median())
df['NO'] = df['NO'].fillna(df['NO'].median())
df['NO2'] = df['NO2'].fillna(df['NO2'].median())
df['Nox'] = df['Nox'].fillna(df['Nox'].median())
df['NH3'] = df['NH3'].fillna(df['NH3'].median())
df['SO2'] = df['SO2'].fillna(df['SO2'].median())
df['CO'] = df['CO'].fillna(df['CO'].median())




# Split into training and test sets
df_x_train, df_x_test, df_y_train, df_y_test = train_test_split(
    df.drop("CO", axis=1),
    df["CO"],
    test_size=0.20,
    #shuffle=False,
    random_state=42,
)

# Create dataframe versions for tabular GAN
df_x_test, df_y_test = df_x_test.reset_index(drop=True), \
  df_y_test.reset_index(drop=True)
df_y_train = pd.DataFrame(df_y_train)
df_y_test = pd.DataFrame(df_y_test)

# Pandas to Numpy
x_train = df_x_train.values
x_test = df_x_test.values
y_train = df_y_train.values
y_test = df_y_test.values

# Build the neural network
model = Sequential()
# Hidden 1
model.add(Dense(50, input_dim=x_train.shape[1], activation='relu'))
model.add(Dense(25, activation='relu')) # Hidden 2
model.add(Dense(12, activation='relu')) # Hidden 2
model.add(Dense(1)) # Output
model.compile(loss='mean_squared_error', optimizer='adam')

monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3,
        patience=5, verbose=1, mode='auto',
        restore_best_weights=True)
model.fit(x_train,y_train,validation_data=(x_test,y_test),
        callbacks=[monitor], verbose=2,epochs=1000)

Epoch 1/1000
827/827 - 2s - loss: 1.0966 - val_loss: 0.1531 - 2s/epoch - 3ms/step
Epoch 2/1000
827/827 - 1s - loss: 0.1543 - val_loss: 0.1265 - 1s/epoch - 2ms/step
Epoch 3/1000
827/827 - 1s - loss: 0.1462 - val_loss: 0.1223 - 1s/epoch - 2ms/step
Epoch 4/1000
827/827 - 1s - loss: 0.1677 - val_loss: 0.1631 - 1s/epoch - 2ms/step
Epoch 5/1000
827/827 - 1s - loss: 0.1452 - val_loss: 0.1425 - 1s/epoch - 2ms/step
Epoch 6/1000
827/827 - 2s - loss: 0.1374 - val_loss: 0.1213 - 2s/epoch - 2ms/step
Epoch 7/1000
827/827 - 2s - loss: 0.1382 - val_loss: 0.1257 - 2s/epoch - 3ms/step
Epoch 8/1000
Restoring model weights from the end of the best epoch: 3.
827/827 - 2s - loss: 0.1264 - val_loss: 0.1412 - 2s/epoch - 2ms/step
Epoch 8: early stopping


In [ ]:
pred = model.predict(x_test)
score = np.sqrt(metrics.mean_squared_error(pred,y_test))
print("Final score (RMSE): {}".format(score))

207/207 [==============================] - 0s 1ms/step
Final score (RMSE): 0.34971615787633487


Training GAN

In [ ]:
from tabgan.sampler import GANGenerator , ForestDiffusionGenerator
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

gen_params = {
    "gen_x_times": 1.1,
    "cat_cols": None,
    "bot_filter_quantile": 0.001,
    "top_filter_quantile": 0.999,
    "is_post_process": True,
    "adversarial_model_params": {
        "metrics": "rmse",
        "max_depth": 2,
        "max_bin": 100,
        "learning_rate": 0.02,
        "random_state": 42,
        "n_estimators": 500,
    },
    "pregeneration_frac": 2,
    "only_generated_data": False,
}


generator = GANGenerator(**gen_params)
gen_x, gen_y = generator.generate_data_pipe(df_x_train, df_y_train, df_x_test)

Fitting CTGAN transformers for each column:   0%|          | 0/8 [00:00<?, ?it/s]

Training CTGAN, epochs::   0%|          | 0/500 [00:00<?, ?it/s]

[LightGBM] [Info] Number of positive: 5291, number of negative: 5291
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000864 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 700
[LightGBM] [Info] Number of data points in the train set: 10582, number of used features: 7
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Number of positive: 5291, number of negative: 5291
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001185 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 700
[LightGBM] [Info] Number of data points in the train set: 10582, number of used features: 7
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Number of positive: 5291, number of negative: 5291
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the

In [ ]:
gen_x

,PM2.5,PM10,NO,NO2,Nox,NH3,SO2
0,343.129428,63.825044,2.776913,1.959922,224.355165,3.728751,36.933029
1,270.881470,93.556723,8.421658,0.926681,254.706466,4.774139,37.246988
2,375.907315,138.850231,156.269082,1.648110,20.388408,12.390049,35.375218
3,360.127431,65.004150,5.317398,3.049897,231.614997,2.318925,38.515018
4,289.245523,141.764729,34.880105,2.120164,215.498022,3.503374,44.230865
...,...,...,...,...,...,...,...
37678,237.360000,450.000000,5.460000,2.380000,5.240000,33.100000,13.780000
37679,238.930000,450.000000,5.550000,2.410000,5.330000,33.400000,15.000000
37680,251.780000,450.000000,5.330000,2.640000,5.230000,15.250000,13.400000
37681,366.570000,450.000000,5.450000,2.680000,5.370000,18.790000,13.470000


In [ ]:
# Predict
pred = model.predict(gen_x.values)
score = np.sqrt(metrics.mean_squared_error(pred,gen_y.values))
print("Final score (RMSE): {}".format(score))

1178/1178 [==============================] - 1s 1ms/step
Final score (RMSE): 1.2071827725087065


In [ ]:

from sklearn.metrics import mean_squared_error

# Predict
pred = model.predict(gen_x.values)
mse = mean_squared_error(gen_y.values, pred)
print("Final score (MSE): {}".format(mse))

1178/1178 [==============================] - 2s 2ms/step
Final score (MSE): 1.4572902462418076


In [ ]:
##############Close#############